In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [16]:
df = pd.read_csv('github-scrapper.csv')
# Strip leading/trailing spaces (if any) from column names
df.columns = df.columns.str.strip()

# Now try to drop the columns again
df = df.drop(['web-scraper-order', 'web-scraper-start-url', 'links', 'links-href', 'pages', 'pages-href'], axis=1)

# Display the updated DataFrame
print(df)


                                                 title  \
0                             upgrade itext dependency   
1                             upgrade itext dependency   
2                             upgrade itext dependency   
3                             upgrade itext dependency   
4                             upgrade itext dependency   
..                                                 ...   
560                  Can't Export PDF, XLS in JRViewer   
561                  Can't Export PDF, XLS in JRViewer   
562                  Can't Export PDF, XLS in JRViewer   
563  Export to PDFs takes significant longer time c...   
564  Export to PDFs takes significant longer time c...   

                                              comments  
0    Hi,\nYou are probably aware that since version...  
1    What about using version 4.2.2?\n<!-- https://...  
2    @teodord\n\nInterested in an PdfBox Implementa...  
3    FYI: I recently got to know about the OpenPDF ...  
4    For the core 

In [17]:
df.to_csv('github-scrapper2.csv', index=False)

In [14]:
dataFrame2 = pd.read_csv('github-scrapper.csv')
dataFrame2.drop(['web-scraper-order', 'web-scraper-start-url', 'links', 'links-href', 'pages', 'pages-href'], axis=1)
print(dataFrame2)

    web-scraper-order                              web-scraper-start-url  \
0        1726468175-1  https://github.com/TIBCOSoftware/jasperreports...   
1        1726468175-2  https://github.com/TIBCOSoftware/jasperreports...   
2        1726468175-3  https://github.com/TIBCOSoftware/jasperreports...   
3        1726468175-4  https://github.com/TIBCOSoftware/jasperreports...   
4        1726468175-5  https://github.com/TIBCOSoftware/jasperreports...   
..                ...                                                ...   
560    1726470653-561  https://github.com/TIBCOSoftware/jasperreports...   
561    1726470653-562  https://github.com/TIBCOSoftware/jasperreports...   
562    1726470653-563  https://github.com/TIBCOSoftware/jasperreports...   
563    1726470668-564  https://github.com/TIBCOSoftware/jasperreports...   
564    1726470668-565  https://github.com/TIBCOSoftware/jasperreports...   

                                                 links  \
0                            

In [127]:
dataFrame1 = pd.read_csv('modified_dataset.csv')
dataFrame1.tail(10)

,title,text,label
72124,An Unlikely Contender Rises in France as the A...,"PARIS — In the age of Donald J. Trump, “Bre...",FAKE
72125,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,REAL
72126,Determined to kill: Can tough gun laws end mas...,The flag at Desert Hot Springs' Condor Gun Sho...,FAKE
72127,WIKILEAKS EMAIL SHOWS CLINTON FOUNDATION FUNDS...,An email released by WikiLeaks on Sunday appea...,REAL
72128,JUDGE JEANINE SOUNDS FREE SPEECH ALARM: “They ...,Judge Jeanine lets it rip! She s concerned wit...,REAL
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,FAKE
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",REAL
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,FAKE
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,FAKE
72133,Goldman Sachs Endorses Hillary Clinton For Pre...,Goldman Sachs Endorses Hillary Clinton For Pre...,REAL


In [128]:
concatenated_rows = pd.concat([dataFrame1, dataFrame2])
concatenated_rows.to_csv('Welfake_politifact_modified.csv', index=False)

In [130]:
data = pd.read_csv('Welfake_politifact_modified.csv')
data.tail()

,title,text,label
93447,NaN,"""I fought to bring about the largest private-s...",REAL
93448,NaN,"""When George Bush said we shouldn't investigat...",REAL
93449,NaN,"Sarah Palin was repeating ""Abraham Lincoln's w...",REAL
93450,NaN,"""He admits he still doesn't know how to use a ...",REAL
93451,NaN,"Obama's one education accomplishment was ""legi...",FAKE


In [131]:
data.fillna(' ', inplace=True)
data.tail()

,title,text,label
93447,,"""I fought to bring about the largest private-s...",REAL
93448,,"""When George Bush said we shouldn't investigat...",REAL
93449,,"Sarah Palin was repeating ""Abraham Lincoln's w...",REAL
93450,,"""He admits he still doesn't know how to use a ...",REAL
93451,,"Obama's one education accomplishment was ""legi...",FAKE


In [132]:
data.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [135]:
data.to_csv('Welfake_politifact_modified.csv', index=False)

In [118]:
# Specify columns to drop
columns_to_drop = ['Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'author', 
                   'created_utc', 'domain', 'hasImage', 'id', 'image_url', 'linked_submission_id', 
                   'num_comments', 'score', 'subreddit', 'title', 'upvote_ratio', '3_way_label', '6_way_label']

# Drop specified columns
data.drop(columns_to_drop, axis=1, inplace=True)

# Rename columns
data.rename(columns={'clean_title': 'text', '2_way_label': 'label'}, inplace=True)

# Replace values in the 'label' column
data['label'] = data['label'].replace({0: 'FAKE', 1: 'REAL'})
data.to_csv('fakeddit_dataset.csv', index=False)
data.tail()

,text,label
970657,clinton campaign uses noise machine to block r...,REAL
970658,a squirrels imprint in wet concrete,REAL
970659,he keeps an eye on the burner whilst i cook,FAKE
970660,video game kingpin super mario is finally capt...,FAKE
970661,mrg,FAKE


In [105]:
label_map = {1: 'REAL', 0: 'FAKE'}
data['label'] = data['label'].map(label_map)
data.drop('Unnamed: 0', axis=1, inplace=True)
data.tail()

,news,label
21313,"""I fought to bring about the largest private-s...",REAL
21314,"""When George Bush said we shouldn't investigat...",REAL
21315,"Sarah Palin was repeating ""Abraham Lincoln's w...",REAL
21316,"""He admits he still doesn't know how to use a ...",REAL
21317,"Obama's one education accomplishment was ""legi...",FAKE


In [107]:
# Rename columns
data.rename(columns={'news': 'text'}, inplace=True)
data.to_csv('politifact_dataset.csv', index=False)
data.tail()


,text,label
21313,"""I fought to bring about the largest private-s...",REAL
21314,"""When George Bush said we shouldn't investigat...",REAL
21315,"Sarah Palin was repeating ""Abraham Lincoln's w...",REAL
21316,"""He admits he still doesn't know how to use a ...",REAL
21317,"Obama's one education accomplishment was ""legi...",FAKE


In [124]:
data = pd.read_csv('modified_dataset.csv')
data.tail()

,title,text,label
72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...,FAKE
72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n...",REAL
72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...,FAKE
72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...,FAKE
72133,Goldman Sachs Endorses Hillary Clinton For Pre...,Goldman Sachs Endorses Hillary Clinton For Pre...,REAL


In [134]:
label_Status = data['label'].value_counts()
transactions = label_Status.index
quantity = label_Status.values

figure = px.pie(data, 
             values=quantity, 
             names=transactions, hole =.60, 
             title="Spam & Ham Status")
figure.show()

In [20]:
# Define the column headers
column_headers = [
    "ID", "label", "statement", "subject(s)", "speaker",
    "speaker's job title", "state info", "party affiliation",
    "barely true counts", "false counts", "half true counts",
    "mostly true counts", "pants on fire counts", "context"
]

# Read the existing TSV file and add the headers
with open('train.tsv', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Insert the headers at the beginning of the file
lines.insert(0, '\t'.join(column_headers) + '\n')

In [1]:
import requests

url = "https://raw.githubusercontent.com/Projects-Developer/Malware-Detection-Using-Deep-Learning-Project/main/Malware%20dataset.csv"
response = requests.get(url)

# Save the content to a file
with open("Malware_dataset.csv", "wb") as file:
    file.write(response.content)

print("Dataset downloaded and saved as Malware_dataset.csv")


Dataset downloaded and saved as Malware_dataset.csv


In [4]:
with open('train_with_headers.tsv', 'w', encoding='utf-8') as f:
    f.writelines(lines)

In [21]:
# Define the new column headers
column_headers = [
    "title", "text", "label"
]

# Read the existing TSV file and extract only required columns
data = []
with open('train.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        columns = line.strip().split('\t')
        data.append([columns[3], columns[2], columns[1]])

# Write the updated content back to the file with new headers
with open('train_modified.tsv', 'w', encoding='utf-8') as f:
    # Write the new headers
    f.write('\t'.join(column_headers) + '\n')
    
    # Write the data rows
    for row in data:
        f.write('\t'.join(row) + '\n')


In [22]:
# Define the new column headers
column_headers = [
    "title", "text", "label"
]

# Read the existing TSV file, modify label values, and extract only required columns
data = []
with open('train.tsv', 'r', encoding='utf-8') as f:
    for line in f:
        columns = line.strip().split('\t')
        label = columns[1]
        # Modify label values
        if "true" in label.lower():
            label = "REAL"
        elif "pants-fire" in label.lower() or "false" in label.lower():
            label = "FAKE"
        data.append('\t'.join([columns[3], columns[2], label]))  # Join columns with tab delimiter

# Write the updated content back to the file with new headers
with open('train_modified.tsv', 'w', encoding='utf-8') as f:
    # Write the new headers
    f.write('\t'.join(column_headers) + '\n')
    
    # Write the data rows
    for row in data:
        f.write(row + '\n')


In [51]:
data = pd.read_csv('all_train.tsv',sep='\t')
data.tail(10)

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,author,clean_title,created_utc,domain,hasImage,id,image_url,linked_submission_id,num_comments,score,subreddit,title,upvote_ratio,2_way_label,3_way_label,6_way_label
878208,878208,878208,179553.0,816673.0,PM_ME_IMSAD,shaolin cat,1.430922e+09,NaN,True,cr07tnf,http://i.imgur.com/i5XIbXJ.jpg,351a9o,NaN,4,psbattle_artwork,Shaolin Cat,NaN,0,2,4
878209,878209,878209,283359.0,1232382.0,nopeitsnotme,silly walks,1.378055e+09,NaN,True,cbzo5cm,http://i.imgur.com/2b6xvHt.jpg,1lil54,NaN,381,psbattle_artwork,Silly Walks,NaN,0,2,4
878210,878210,878210,NaN,NaN,Jackrymer21,the sky was dabbing on me today,1.563758e+09,i.redd.it,True,cg6nmo,https://preview.redd.it/aloml4bv8rb31.jpg?widt...,NaN,4.0,4,mildlyinteresting,The sky was dabbing on me today,0.56,1,0,0
878211,878211,878211,NaN,NaN,apieracc,found this guy after moving my shower caddy of...,1.365028e+09,i.imgur.com,True,1bmb6h,https://external-preview.redd.it/3pcutlT6hYUy7...,NaN,0.0,5,pareidolia,Found this guy after moving my shower caddy of...,1.00,0,2,2
878212,878212,878212,NaN,NaN,wesman212,hello kitty arrested on drunken driving charge,1.414604e+09,wtnh.com,False,2kozsa,NaN,NaN,0.0,6,nottheonion,‘Hello Kitty’ arrested on drunken driving charge,0.64,1,0,0
878213,878213,878213,NaN,NaN,PotatoSlapper,clinton campaign uses noise machine to block r...,1.460096e+09,dailycaller.com,False,4dv6j8,NaN,NaN,1.0,7,nottheonion,Clinton Campaign Uses Noise Machine To Block R...,1.00,1,0,0
878214,878214,878214,NaN,NaN,fatkiddown,a squirrels imprint in wet concrete,1.472570e+09,i.redd.it,True,50bmkc,https://preview.redd.it/mhph05cnveix.jpg?width...,NaN,1.0,9,photoshopbattles,PsBattle: A Squirrel's Imprint In Wet Concrete...,0.90,1,0,0
878215,878215,878215,NaN,NaN,MrChrisOD,he keeps an eye on the burner whilst i cook,1.520807e+09,i.imgur.com,True,83q3ot,https://external-preview.redd.it/afTHQfSIpVY52...,NaN,0.0,2,pareidolia,He keeps an eye on the burner whilst I cook,0.75,0,2,2
878216,878216,878216,NaN,NaN,NaN,video game kingpin super mario is finally capt...,1.565283e+09,i.redd.it,False,cnoe3v,NaN,NaN,0.0,16,fakehistoryporn,"Video Game kingpin ""Super Mario"" is finally ca...",1.00,0,2,2
878217,878217,878217,62754.0,280950.0,GeneralFelixBraxton,mrg,1.505828e+09,NaN,True,dn7ou9u,https://i.imgur.com/IiARoWE.jpg,70w3kt,NaN,1,psbattle_artwork,Mr.G,NaN,0,2,4


In [81]:
data = pd.read_csv('train.csv')
data.tail(10)

,Unnamed: 0,news,label
17080,17080,"McCain economic adviser Phil Gramm is ""the arc...",1
17081,17081,"""I have never asked for a single earmark, pork...",0
17082,17082,John McCain said last year he didn't know of a...,1
17083,17083,"""Obama has asked for $932-million in earmarks,...",1
17084,17084,"""Through competition as governor, I got agreem...",0
17085,17085,"""I fought to bring about the largest private-s...",1
17086,17086,"""When George Bush said we shouldn't investigat...",1
17087,17087,"Sarah Palin was repeating ""Abraham Lincoln's w...",1
17088,17088,"""He admits he still doesn't know how to use a ...",1
17089,17089,"Obama's one education accomplishment was ""legi...",0


In [69]:
# Rename columns
data.rename(columns={'verdict': 'label', 'statement': 'text'}, inplace=True)

# Replace values in the 'label' column
data['label'] = data['label'].replace({'false': 'FAKE', 'mostly-false': 'FAKE', 'pants-fire': 'FAKE',
                                       'half-true': 'REAL', 'mostly-true': 'REAL', 'true': 'REAL'})

# Select only the required columns
data = data[['label', 'text']]

# Display the first few rows of the modified DataFrame
print(data.head())

  label                                               text
0  REAL  John McCain opposed bankruptcy protections for...
1  FAKE  "Bennie Thompson actively cheer-led riots in t...
2  REAL  Says Maggie Hassan was "out of state on 30 day...
3  FAKE  "BUSTED: CDC Inflated COVID Numbers, Accused o...
4  REAL  "I'm the only (Republican) candidate that has ...


In [82]:
data.isnull().sum()

Unnamed: 0    0
news          0
label         0
dtype: int64

In [83]:
label_Status = data['label'].value_counts()
transactions = label_Status.index
quantity = label_Status.values

figure = px.pie(data, 
             values=quantity, 
             names=transactions, hole =.60, 
             title="Spam & Ham Status")
figure.show()

In [3]:
import pandas as pd
from transformers import DistilBertTokenizer

# Load the dataset
df = pd.read_csv('WelFake_Dataset.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.fillna(' ', inplace=True)
features = df[['text']]

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to calculate token lengths
def calculate_token_length(text, tokenizer):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    return len(tokens)

# Apply the function to the 'text' column
df['token_length'] = df['text'].apply(lambda x: calculate_token_length(x, tokenizer))

# Display the first few rows to verify
print(df.head())

Token indices sequence length is longer than the specified maximum sequence length for this model (1200 > 512). Running this sequence through the model will result in indexing errors


                                               title  \
0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1                                                      
2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3  Bobby Jindal, raised Hindu, uses story of Chri...   
4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  token_length  
0  No comment is expected from Barack Obama Membe...      1          1200  
1     Did they post their votes for Hillary already?      1            11  
2   Now, most of the demonstrators gathered last ...      1            38  
3  A dozen politically active pastors came here f...      0          1646  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1           425  


In [2]:
import pandas as pd

# Assuming you've already read the CSV file into a DataFrame
data = pd.read_csv('Carbon emission/ghg-emissions2.csv')

# Drop the columns for the years 2020, 2021, and 2022
data = data.drop(columns=['2020', '2021', '2022'])

# Display the resulting DataFrame
print(data)

           iso                                     Country/Region    unit  \
0          BGD                                         Bangladesh  MtCO2e   
1  Data source                                                PIK     NaN   
2     Data for  Location: Bangladesh. Sectors/Subsectors: Tota...     NaN   

   1850  1851  1852  1853  1854  1855  1856  ...   2010   2011   2012   2013  \
0  21.5  21.7  22.0  22.2  22.4  22.6  22.8  ...  139.0  144.0  150.0  152.0   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN    NaN    NaN    NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN    NaN    NaN    NaN   

    2014   2015   2016   2017   2018   2019  
0  158.0  168.0  171.0  179.0  188.0  201.0  
1    NaN    NaN    NaN    NaN    NaN    NaN  
2    NaN    NaN    NaN    NaN    NaN    NaN  

[3 rows x 173 columns]


In [4]:
data.to_csv('Carbon emission/ghg-emissions2.csv', index=False)